In [61]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import demoji

# Baixe recursos adicionais do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\higor\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [62]:
df = pd.read_csv('data/bolsonaro_tweets.csv')
df.head()

,date,text,likes,retweets,link
0,2020-10-21,- Não se justifica um bilionário aporte financ...,16890,2519,https://twitter.com/jairbolsonaro/status/13189...
1,2020-10-21,A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...,23349,4174,https://twitter.com/jairbolsonaro/status/13189...
2,2020-10-21,@secomvc @ItamaratyGovBr @MinEconomia @ernesto...,2874,154,https://twitter.com/jairbolsonaro/status/13188...
3,2020-10-21,- Recebi a visita do Conselheiro de Segurança ...,19593,3391,https://twitter.com/jairbolsonaro/status/13188...
4,2020-10-20,"- CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...",16221,3227,https://twitter.com/jairbolsonaro/status/13185...


In [63]:
# Verificando se há valores nulos
print(df.isnull().sum())
# Verificando o tipo de dado de cada coluna
print(df.dtypes)
# Descrição do dataset
print(df.shape)

date        7225
text           0
likes          0
retweets       0
link           0
dtype: int64
date        object
text        object
likes        int64
retweets     int64
link        object
dtype: object
(9945, 5)


In [64]:
# Converter a coluna 'date' para o tipo datetime
df['date'] = pd.to_datetime(df['date'])
# Preencher com a data anterior
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['date'] = df['date'].ffill() # forward fill

print(df.isnull().sum())


date        0
text        0
likes       0
retweets    0
link        0
dtype: int64


In [65]:
# Tokenizar os tweets

# Tokenização e pré-processamento
stop_words = set(stopwords.words('portuguese'))
ps = PorterStemmer()

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [token for token in tokens if token.isalnum() and token not in stop_words]
    tokens = [ps.stem(token) for token in tokens]
    return tokens

# Aplicar a função de pré-processamento aos tweets
df['tokens'] = df['text'].apply(preprocess_text)

# Visualizar o resultado
print(df[['text', 'tokens']].head())

                                                text  \
0  - Não se justifica um bilionário aporte financ...   
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...   
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...   
3  - Recebi a visita do Conselheiro de Segurança ...   
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...   

                                              tokens  
0  [justifica, bilionário, aport, financeiro, med...  
1  [vacina, chinesa, joão, doria, governo, qualqu...  
2  [secomvc, itamaratygovbr, mineconomia, ernesto...  
3  [recebi, visita, conselheiro, segurança, eua, ...  
4  [curso, técnico, portaria, autoriza, oferta, 5...  


In [66]:
# Extrair menções usando regex
df['mencoes'] = df['text'].apply(lambda x: re.findall(r'@(\w+)', x))

# Visualizar o resultado
print(df[['text', 'mencoes']].head())

                                                text  \
0  - Não se justifica um bilionário aporte financ...   
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...   
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...   
3  - Recebi a visita do Conselheiro de Segurança ...   
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...   

                                             mencoes  
0                                                 []  
1                                                 []  
2  [secomvc, ItamaratyGovBr, MinEconomia, ernesto...  
3                                                 []  
4                     [MEC_Comunicacao, mribeiroMEC]  


In [67]:
# Função para extrair e contar hashtags de um tweet
def extrai_hashtags(text):
    hashtags = re.findall(r'#(\w+)', text)
    return hashtags

# Criar uma nova coluna 'hashtags' no DataFrame
df['hashtags'] = df['text'].apply(extrai_hashtags)

# Visualizar o resultado
print(df[['text', 'hashtags']].head())

                                                text hashtags
0  - Não se justifica um bilionário aporte financ...       []
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...       []
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...       []
3  - Recebi a visita do Conselheiro de Segurança ...       []
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...       []


In [68]:
# Função para extrair emojis de um texto
def extract_emojis(text):
    return demoji.findall(text)

# Criar uma nova coluna 'emojis' no DataFrame
df['emojis'] = df['text'].apply(extract_emojis)

# Visualizar o resultado
print(df[['text', 'emojis']].head())

                                                text              emojis
0  - Não se justifica um bilionário aporte financ...                  {}
1  A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...                  {}
2  @secomvc @ItamaratyGovBr @MinEconomia @ernesto...  {'🤝': 'handshake'}
3  - Recebi a visita do Conselheiro de Segurança ...                  {}
4  - CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...                  {}


In [69]:
df.head()

,date,text,likes,retweets,link,tokens,mencoes,hashtags,emojis
0,2020-10-21,- Não se justifica um bilionário aporte financ...,16890,2519,https://twitter.com/jairbolsonaro/status/13189...,"[justifica, bilionário, aport, financeiro, med...",[],[],{}
1,2020-10-21,A VACINA CHINESA DE JOÃO DORIA\n- Para o meu G...,23349,4174,https://twitter.com/jairbolsonaro/status/13189...,"[vacina, chinesa, joão, doria, governo, qualqu...",[],[],{}
2,2020-10-21,@secomvc @ItamaratyGovBr @MinEconomia @ernesto...,2874,154,https://twitter.com/jairbolsonaro/status/13188...,"[secomvc, itamaratygovbr, mineconomia, ernesto...","[secomvc, ItamaratyGovBr, MinEconomia, ernesto...",[],{'🤝': 'handshake'}
3,2020-10-21,- Recebi a visita do Conselheiro de Segurança ...,19593,3391,https://twitter.com/jairbolsonaro/status/13188...,"[recebi, visita, conselheiro, segurança, eua, ...",[],[],{}
4,2020-10-20,"- CURSOS TÉCNICOS. \n- Portaria 524-MEC, de 30...",16221,3227,https://twitter.com/jairbolsonaro/status/13185...,"[curso, técnico, portaria, autoriza, oferta, 5...","[MEC_Comunicacao, mribeiroMEC]",[],{}


In [76]:
# Ver as 25 menções mais frequentes
mencoes = df['mencoes'].explode().value_counts() # explode() cria uma linha para cada menção
print(mencoes[:25])
# Ministerios mais mencionados: MinInfraestrutura (Tarcísio), MinEconomia (Guedes), MinSaúde (Mandetta/Teich/Pazuello)
# Filhos mais mencionados: Carlos, Eduardo, Flávio 

mencoes
CarlosBolsonaro    354
BolsonaroSP        279
govbr              208
MInfraestrutura    203
jairbolsonaro      191
tarcisiogdf        183
FlavioBolsonaro    160
MinEconomia        138
secomvc            119
minsaude           114
mdregional_br      111
exercitooficial     79
SF_Moro             76
DNIToficial         75
MEC_Comunicacao     70
odiodobem           60
tvbrasilgov         54
DefesaGovBr         53
ernestofaraujo      49
rogeriosmarinho     48
BlogDoPim           47
MinCidadania        47
TerezaCrisMS        46
DamaresAlves        43
PRFBrasil           42
Name: count, dtype: int64


In [79]:
# HashTags mais frequentes
hashtags = df['hashtags'].explode().value_counts()
print(hashtags[:10]) # Destaque para hashtags de apoio a si mesmo e de afrontração a opositores

hashtags
NasRuasComBolsonaro    19
choramais              10
Brasil                  8
fakenews                5
BolsonaroNaBand         5
Projetoanticrime        4
PPI                     4
90Minutos               4
tbt                     3
TodosComBolsonaro       3
Name: count, dtype: int64


In [80]:
# Emojis mais frequentes
emojis = df['emojis'].explode().value_counts()
print(emojis[:10]) # Destaque para emojis com a bandeira do Brasil e de comunicação "informal", destaque para a maior frequencia para a bandeira dos EUA

emojis
🇧🇷    735
👍🏻    392
👍     294
🤝     146
😂      50
😍      49
👉🏻     47
🇺🇸     23
👍🏼     22
🤔      19
Name: count, dtype: int64


In [70]:
# Exportar o DataFrame para um arquivo CSV e selecionar as colunas de interesse
df[['date', 'tokens','likes', 'retweets', 'mencoes', 'hashtags', 'emojis']].to_csv('data/bolsonaro_tweets_preprocessado.csv', index=False)